In [1]:
import os
os.sys.path.append('../')
import data.data_loader as DL

In [2]:
SITW_home_dir = '/home/ideal_kar/Dataset/sitw_database.v4/'
SITW_dev_dir = os.path.join(SITW_home_dir, 'dev/')
SITW_eval_dir = os.path.join(SITW_home_dir, 'eval/')
SITW_dev_key_dir = os.path.join(SITW_dev_dir, 'key/')
SITW_eval_key_dir = os.path.join(SITW_eval_dir, 'key/')

In [3]:
temp_audio = DL.load_audio(SITW_dev_dir + 'audio/zzpjc.flac')

In [3]:
def read_meta_info(mode, SITW_home_dir, testPOI=None):
    
    if mode is 'dev':
        SITW_dev_dir = os.path.join(SITW_home_dir, 'dev/')
        SITW_eval_dir = None
    elif mode is 'eval':
        SITW_dev_dir = None
        SITW_eval_dir = os.path.join(SITW_home_dir, 'eval/')
    elif mode is 'all':
        SITW_dev_dir = os.path.join(SITW_home_dir, 'dev/')
        SITW_eval_dir = os.path.join(SITW_home_dir, 'eval/')
    else:
        raise BaseException("Invalid mode")
        
    if testPOI is None:
        testPOI = []
    
    file_dirs, spk_IDs, genders, sess_IDs, mic_types, spk_nums, artifacts, levels, rec_envs, metadata \
    = [], [], [], [], [], [], [], [], [], []
    
    file_dirs_v, spk_IDs_v, genders_v, sess_IDs_v, mic_types_v, spk_nums_v, artifacts_v, levels_v, rec_envs_v, metadata_v \
    = [], [], [], [], [], [], [], [], [], []
    
    SITW_dirs = [SITW_dev_dir, SITW_eval_dir]
    count = 0
    for SITW_dir in SITW_dirs:
        if SITW_dir is None:
            continue
        with open(SITW_dir + 'keys/meta.lst', 'r') as f:
            lines = f.readlines()
            
            for line in lines:
                count += 1
                split_line = line.strip().split(' ')
                if split_line[1][0] in testPOI:
                    file_dirs_v.append(SITW_dir + split_line[0])
                    spk_IDs_v.append(split_line[1])
                    genders_v.append(split_line[2])
                    sess_IDs_v.append(split_line[4])
                    mic_types_v.append(split_line[3])
                    spk_nums_v.append(split_line[7])
                    artifacts_v.append(split_line[8])
                    levels_v.append(split_line[9])
                    rec_envs_v.append(split_line[10])
                    try:
                        metadata_v.append(split_line[11])
                    except:
                        metadata_v.append(None)
                else:
                    file_dirs.append(SITW_dir + split_line[0])
                    spk_IDs.append(split_line[1])
                    genders.append(split_line[2])
                    sess_IDs.append(split_line[4])
                    mic_types.append(split_line[3])
                    spk_nums.append(split_line[7])
                    artifacts.append(split_line[8])
                    levels.append(split_line[9])
                    rec_envs.append(split_line[10])
                    try:
                        metadata.append(split_line[11])
                    except:
                        metadata.append(None)
    print(count)
    return [file_dirs, spk_IDs, genders, sess_IDs, mic_types, spk_nums, artifacts, levels, rec_envs, metadata], \
            [file_dirs_v, spk_IDs_v, genders_v, sess_IDs_v, mic_types_v, spk_nums_v, artifacts_v, levels_v, rec_envs_v, metadata_v]

In [4]:
def spk_ID_to_label(spk_IDs):
    label = []
    mapping = {}
    count = 0
    for spk_ID in spk_IDs:
        if spk_ID not in mapping:
            label.append(count)
            mapping[spk_ID] = count
            count += 1
        else:
            label.append(mapping[spk_ID])
    '''
    @return vars
    label: list type, contains numerical labels for whole spk_IDs 
    mapping: dict type, contains mapping between actual speaker & numerical label
    '''
    return label, mapping

In [38]:
SITW_lists, _ = read_meta_info('all', SITW_home_dir)
total_label_mapping = spk_ID_to_label(SITW_lists[1])[1]
#print(spk_label_mapping)
mapping = {}
for spk_ID in total_label_mapping.keys():
    
    if spk_ID[0] not in mapping:
        mapping[spk_ID[0]] = 1
    else:
        mapping[spk_ID[0]] += 1
        
print(mapping)
        

4841
{'Q': 15, 'C': 15, 'Y': 14, 'O': 8, 'E': 11, 'V': 13, 'I': 16, 'A': 17, 'S': 8, 'P': 7, 'B': 18, 'Z': 7, 'D': 13, 'L': 11, 'R': 8, 'U': 6, 'T': 15, 'F': 15, 'N': 8, 'K': 17, 'M': 12, 'J': 11, 'G': 16, 'W': 12, 'H': 4, 'X': 2}


In [29]:
def SITW_make_manifest(mode, SITW_home_dir, testPOI):
    iden_lists, veri_lists = read_meta_info(mode, SITW_home_dir, testPOI)
    
    iden_label, iden_label_mapping = spk_ID_to_label(iden_lists[1])
    
    shuffle_index = shuffle(range(len(iden_lists[0])))
    iden_lists = [[iden_list[j] for j in shuffle_index] for iden_list in iden_lists]
    
    shuffle_index = shuffle(range(len(veri_lists[0])))
    veri_lists = [[veri_list[j] for j in shuffle_index] for veri_list in veri_lists]
    
    #print(len(iden_lists[0]), len(veri_lists[0]))
    
    iden_len = len(iden_lists[0])
    train_lim = int(iden_len * 0.7)
    val_lim = int(iden_len * 0.8)
    
    iden_datasets = {'train': zip(iden_lists[0][:train_lim], iden_label[:train_lim]), \
                     'val': zip(iden_lists[0][train_lim:val_lim], iden_label[train_lim:val_lim]), \
                     'test': zip(iden_lists[0][val_lim:], iden_label[val_lim:])}
    
    for dataset in iden_datasets.keys():
        with open("../data/SITW_iden_{}_manifest.csv".format(dataset), "w") as f:
            for file_dir, label in iden_datasets[dataset]:
                #print(dataset, len(iden_data))
                f.write(file_dir + ',' + str(label))
                f.write(os.linesep)
        f.close()
        
    veri_train_c = list(itertools.combinations(iden_lists[0][:int(iden_len / 4)], 2))
    veri_train_spk_c = list(itertools.combinations(iden_lists[1][:int(iden_len / 4)], 2))
    veri_test_c = list(itertools.combinations(veri_lists[0][:int(len(veri_lists[0])/2)], 2))
    veri_test_spk_c = list(itertools.combinations(veri_lists[1][:int(len(veri_lists[1])/2)], 2))

    veri_datasets = {'train': zip(veri_train_c, veri_train_spk_c), 'test': zip(veri_test_c, veri_test_spk_c)}
    print(len(veri_train_c), len(veri_train_spk_c))
    print(len(veri_test_c), len(veri_test_spk_c))
    
    for dataset in veri_datasets.keys():
        one_num, zero_num = 0, 0
        with open("../data/SITW_veri_{}_manifest.csv".format(dataset), "w") as f:
            for dirs, spks in veri_datasets[dataset]:
                line = dirs[0] + ',' + dirs[1] + ','
                if spks[0] == spks[1]:
                    line = line + '1'
                    one_num += 1
                else:
                    line = line + '0'
                    zero_num += 1
                    
                f.write(line + os.linesep)
        f.close()
        print(one_num, zero_num)

In [30]:
from sklearn.utils import shuffle
import itertools
SITW_make_manifest('all', SITW_home_dir, ['A', 'B', 'C', 'D'])

446040 446040
140185 140185
1929 444111
2247 137938
